In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import cv2
import numpy as np
import shutil
import os
import matplotlib.pyplot as plt
from time import sleep
from IPython.display import Image, display
from PIL import Image

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
EfficientNetV2_path = "C:/Users/jamay/OneDrive/Desktop/EfficientNetV2_S&SGD"
first_model= torch.load(EfficientNetV2_path, map_location=device)
first_model.eval()

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  

In [4]:
def predict_fire(model, image):
    resized_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = np.array(resized_image)
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    def preprocess_image(image):
        image = transform(image)
        image = image.unsqueeze(0)
        return image.to(device)
    test_image = preprocess_image(image)
    with torch.no_grad():
        test_image = test_image.to(device)
        outputs = model(test_image)
        probabilities = F.softmax(outputs, dim=1)
        predicted_probability = probabilities[0][0].item()
    return predicted_probability

In [5]:
threshold_alarm = 0.95
input_height = 224
input_width = 224
red = (0, 0, 255)
green = (0, 255, 0)

In [6]:
def list_mp4_videos_in_folder(folder_path):
    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter for video files with ".mp4" extension
    mp4_video_names = [file for file in files if file.lower().endswith('.mp4')]

    return mp4_video_names

# Replace 'your_folder_path' with the path to your video folder
folder_path = "C:/Users/jamay/OneDrive/Desktop/wildfire train videos"
mp4_video_names = list_mp4_videos_in_folder(folder_path)

In [7]:
mp4_video_names

['20170807-FIRE-bh-n-mobo-c.mp4',
 '20180504-FIRE-smer-tcs10-mobo-c.mp4',
 '20180602-Alison-sp-w-mobo-c.mp4',
 '20180727-FIRE-mg-w-mobo-c.mp4',
 '20180806-Holy-sp-s-mobo-c.mp4',
 '20180809-FIRE-mg-w-mobo-c.mp4',
 '20180813-FIRE-bh-s-mobo-c.mp4',
 '20180813-FIRE-bl-n-mobo-c.mp4',
 '20180827_Holy_Fire_flareup_early_phase_near_Santiago_Pe_YOXAfXOGkWo_137.mp4',
 '20190529-94Fire-lp-s-mobo-c.mp4',
 '20190529-94Fire-om-n-mobo-c.mp4',
 '20190610-FIRE-bh-w-mobo-c.mp4',
 '20190610-Pauma-bh-w-mobo-c.mp4',
 '20190714-PinosSouth-pi-s-mobo-c.mp4',
 '20190716-FIRE-bl-s-mobo-c.mp4',
 '20190716-FIRE-mg-n-mobo-c.mp4',
 '20190716-Meadowfire-hp-n-mobo-c.mp4',
 '20190717-FIRE-lp-n-mobo-c.mp4',
 '20190728-Dehesa-lp-n-mobo.mp4',
 '20190728-FIRE-om-n-mobo-c.mp4',
 '20190803-OtaySouth-lp-s-mobo.mp4',
 '20190803-OtaySouth-om-s-mobo.mp4',
 '20190809-PinosSouth-pi-s-mobo.mp4',
 '20190810-SantaFire-rm-w-mobo.mp4',
 '20190813-FIRE-69bravo-e-mobo-c.mp4',
 '20190813-Topanga-69bravo-n-mobo.mp4',
 '20190814-Border-lp-

In [12]:
def list_videos_in_folder(folder_path):
    video_list = []

    # List all files in the folder
    files = os.listdir(folder_path)

    # Filter for video files (you can customize the extensions)
    video_extensions = ['.mp4', '.avi', '.mkv', '.mov', '.wmv']
    video_list = [file for file in files if any(file.lower().endswith(ext) for ext in video_extensions)]

    # Extract video names without extension
    video_names = [os.path.splitext(video)[0] for video in video_list]

    return video_names

# Replace 'your_folder_path' with the path to your video folder
folder_path = "C:/Users/jamay/OneDrive/Desktop/wildfire train videos"
video_names = list_videos_in_folder(folder_path)

In [13]:
video_names

['20170807-FIRE-bh-n-mobo-c',
 '20180504-FIRE-smer-tcs10-mobo-c',
 '20180602-Alison-sp-w-mobo-c',
 '20180727-FIRE-mg-w-mobo-c',
 '20180806-Holy-sp-s-mobo-c',
 '20180809-FIRE-mg-w-mobo-c',
 '20180813-FIRE-bh-s-mobo-c',
 '20180813-FIRE-bl-n-mobo-c',
 '20180827_Holy_Fire_flareup_early_phase_near_Santiago_Pe_YOXAfXOGkWo_137',
 '20190529-94Fire-lp-s-mobo-c',
 '20190529-94Fire-om-n-mobo-c',
 '20190610-FIRE-bh-w-mobo-c',
 '20190610-Pauma-bh-w-mobo-c',
 '20190714-PinosSouth-pi-s-mobo-c',
 '20190716-FIRE-bl-s-mobo-c',
 '20190716-FIRE-mg-n-mobo-c',
 '20190716-Meadowfire-hp-n-mobo-c',
 '20190717-FIRE-lp-n-mobo-c',
 '20190728-Dehesa-lp-n-mobo',
 '20190728-FIRE-om-n-mobo-c',
 '20190803-OtaySouth-lp-s-mobo',
 '20190803-OtaySouth-om-s-mobo',
 '20190809-PinosSouth-pi-s-mobo',
 '20190810-SantaFire-rm-w-mobo',
 '20190813-FIRE-69bravo-e-mobo-c',
 '20190813-Topanga-69bravo-n-mobo',
 '20190814-Border-lp-s-mobo',
 '20190814-FIRE-om-e-mobo-c',
 '20190814-FIRE-pi-s-mobo-c',
 '20190829-FIRE-smer-tcs8-mobo-c',


In [14]:
base_path = "C:/Users/jamay/OneDrive/Desktop/wildfire dataset"
def process_video(video_name):

    ret, image1 = name.read()
    image1 = cv2.resize(image1, (1920, 1080))
    mean = []
    frame_number = 0

    while True:
        image2 = image1
        ret, image1 = name.read()
        frame_number += 1

        if not ret:
            break

        print(f"Frame number {frame_number} has been processed.")

        image1 = cv2.resize(image1, (1920, 1080))
        img_draw = image1.copy()

        for m in range(5):
            for n in range(9):

                original_image1 = cv2.resize(image1[m * 180:m * 180 + 359, n * 180 + 60:n * 180 + 359 + 60, :], (input_height, input_width))
                original_image2 = cv2.resize(image2[m * 180:m * 180 + 359, n * 180 + 60:n * 180 + 359 + 60, :], (input_height, input_width))

                results_Fire = predict_fire(first_model, original_image1)

                if results_Fire > threshold_alarm:

                    filename_img1 = f'{video_name}_frame_{frame_number}_block_{m}_{n}.png'
                    directory_path = f'{base_path}/A'
                    file_path = os.path.join(directory_path, filename_img1)
                    cv2.imwrite(file_path, original_image2)
                    #plt.imshow(cv2.cvtColor(original_image2, cv2.COLOR_BGR2RGB))
                    #plt.title(f'{video_name}, Frame {frame_number}, Block ({m},{n})')
                    #plt.show()

                    filename_img2 = f'{video_name}_frame_{frame_number+1}_block_{m}_{n}.png'
                    directory_path = f'{base_path}/B'
                    file_path = os.path.join(directory_path, filename_img2)
                    cv2.imwrite(file_path, original_image1)
                    #plt.imshow(cv2.cvtColor(original_image1, cv2.COLOR_BGR2RGB))
                    #plt.title(f'{video_name}, Frame {frame_number+1}, Block ({m},{n})')
                    #plt.show()


    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [15]:
base_video_path = "C:/Users/jamay/OneDrive/Desktop/wildfire train videos"

In [16]:
for video_index in range(0, 48):
    video_name = video_names[video_index]
    name = mp4_video_names[video_index]
    video_path = os.path.join(base_video_path, name)
    name = cv2.VideoCapture(video_path)
    print(f"Video number {video_index} with name {video_name} has been started.")
    process_video(video_name)

Video number 0 with name 20170807-FIRE-bh-n-mobo-c has been started.
Frame number 1 has been processed.
Frame number 2 has been processed.
Frame number 3 has been processed.
Frame number 4 has been processed.
Frame number 5 has been processed.
Frame number 6 has been processed.
Frame number 7 has been processed.
Frame number 8 has been processed.
Frame number 9 has been processed.
Frame number 10 has been processed.
Frame number 11 has been processed.
Frame number 12 has been processed.
Frame number 13 has been processed.
Frame number 14 has been processed.
Frame number 15 has been processed.
Frame number 16 has been processed.
Frame number 17 has been processed.
Frame number 18 has been processed.
Frame number 19 has been processed.
Frame number 20 has been processed.
Frame number 21 has been processed.
Frame number 22 has been processed.
Frame number 23 has been processed.
Frame number 24 has been processed.
Frame number 25 has been processed.
Frame number 26 has been processed.
Fram

KeyboardInterrupt: 